In [81]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_binary

In [94]:
driver = webdriver.Chrome()
rent_category = []
driver.get('https://www.f-takken.com/freins/rent/houses/area/items?listtype=houses&lang=ja&location=area&locate%5B0%5D=40131&locate%5B1%5D=40132&locate%5B2%5D=40133&locate%5B3%5D=40134&locate%5B4%5D=40135&locate%5B5%5D=40136&locate%5B6%5D=40137&order1=nn&order2=&limit=20&data_21=0&data_22=0&data_30=0&data_31=0&data_486=&data_486_code=&data_487=&data_487_code=&page=1&currentTabIndex=0')
time.sleep(5)

# 検索先のページのHTMLを取得
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

# 表示されているカテゴリの番号を全て記憶 
for element in soup.find_all("li",class_ = "item list-tpl"):
    data_id = element.get("data-id")
    rent_category.append(data_id)
    time.sleep(5)

print(rent_category)

# 1000個のカテゴリ番号を保存できるまで、次のページに遷移する












['98849257', '98832815', '99177351', '99317728', '99272715', '99306355', '99314815', '99279318', '99226734', '99309879', '99307676', '99309396', '99314876', '99298634', '99290130', '99316614', '99294643', '99316611', '99316642', '99299022']


In [74]:
# ページ遷移
driver.get("https://www.f-takken.com/freins/items/{}".format(search_box))